In [112]:
import re
import sys
import nltk
import string
from math import sqrt, log
from collections import defaultdict
from itertools import chain, product
from nltk import word_tokenize as tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer as stemmer
import numpy as np
from itertools import groupby
from collections import Counter
from nltk.collocations import *
from nltk.stem.porter import PorterStemmer
import pandas as pd
import textfeatures as tf
from sklearn import preprocessing
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from textstat.textstat import textstatistics, easy_word_set, legacy_round

ModuleNotFoundError: No module named 'textstat'

In [121]:
def compute_average_word_length(sentence):
    return np.mean([len(words) for words in sentence.split()])

def compute_average_sentence_length(sentence):
    sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", sentence)
    return np.mean([len(words) for words in sentence])

def freq_of_words_great_sent_len(sentence):
    result = []
    avg_word_len = compute_average_word_length(sentence)
    # sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", sentence)
    sentence = Counter(sentence.split())
    for key, value in sentence.items():
        if len(key) > avg_word_len:
            result.append(value)
#             print (key, value)
    return sum(result)

def tokenize(sentence):
    return re.split(r"[^0-9A-Za-z\-'_]+", sentence)

def compute_yules_k_for_text(sentence):
    tokens = tokenize(sentence)
    counter = Counter(token.upper() for token in tokens)

    #compute number of word forms in a given sentence/text
    m1 = sum(counter.values())
    m2 = sum([frequency ** 2 for frequency in counter.values()])

    #compute yules k measure and return the value
    yules_k = 10000/((m1 * m1) / (m2 - m1))
    return yules_k


def words_in_sentence(sentence):
    w = [words.strip("0123456789!:,.?()[]{}") for words in sentence.split()]
    return filter(lambda x: len(x) > 0, w)

def compute_yules_i_for_text(sentence):
    dictionary = {}
    stemmer = PorterStemmer()

    for word in words_in_sentence(sentence):
        word = stemmer.stem(word).lower()
        try:
            dictionary[word] += 1
        except:
            dictionary[word] = 1

    m1 = float(len(dictionary))
    m2 = sum([len(list(grouped_values)) * (frequency ** 2) for frequency, grouped_values in groupby(sorted(dictionary.values()))])

    # compute yules i and return the value
    try:
        yules_i = (m1 * m1) / (m2 - m1)
        return yules_i
    except ZeroDivisionError:
        return 0
    
# polysemy for the content words
def polysemy(group):
    stop = stopwords.words('english')
    str1 = [i for i in group.split() if i not in stop]
    a = list()
    for w in str1:
        if(len(wn.synsets(w)) > 1):
            a.append(w)
    return len(a)

# Returns average sentence length
def avg_sentence_length(self, text):
    words = word_count(text)
    sentences = sentence_count(text)
    average_sentence_length = float(words / sentences)
    return average_sentence_length
def syllables_count(word):
    return textstatistics().syllable_count(word)
# Returns the average number of syllables per
# word in the text
def avg_syllables_per_word(text):
    syllable = syllables_count(text)
    words = word_count(text)
    ASPW = float(syllable) / float(words)
def break_sentences(text):
    nlp = spacy.load('en')
    doc = nlp(text)
    return doc.sents
# Returns Number of Words in the text
def word_count(text):
    sentences = break_sentences(text)
    words = 0
    for sentence in sentences:
        words += len([token for token in sentence])
    return words
# Returns the number of sentences in the text
def sentence_count(text):
    sentences = break_sentences(text)

def flesch_reading_ease(text):
    """
        Implements Flesch Formula:
        Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW)
        Here,
          ASL = average sentence length (number of words
                divided by number of sentences)
          ASW = average word length in syllables (number of syllables
                divided by number of words)
    """
    FRE = 206.835 - float(1.015 * avg_sentence_length(text)) -\
          float(84.6 * avg_syllables_per_word(text))
    return legacy_round(FRE, 2)

In [122]:
raw_data = pd.read_csv("scores.csv")
# raw_data = raw_data.sample(frac=0.01).reset_index(drop=True)
raw_data = raw_data.sample(frac=1).reset_index(drop=True)

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    return content

In [125]:
raw_data["yules_i_measure_abs"] = np.nan
raw_data["yules_i_measure_abs"] = raw_data.apply(lambda x: compute_yules_i_for_text(x['abstract']), axis=1)

raw_data["avg_word_len_abs"] = np.nan
raw_data["avg_word_len_abs"] = raw_data.apply(lambda x: compute_average_word_length(x['abstract']), axis=1)

raw_data["avg_sen_len_abs"] = np.nan
raw_data["avg_sen_len_abs"] = raw_data.apply(lambda x: compute_average_sentence_length(x['abstract']), axis=1)

raw_data["freq_words_gt_sen_len_abs"] = np.nan
raw_data["freq_words_gt_sen_len_abs"] = raw_data.apply(lambda x: freq_of_words_great_sent_len(x['abstract']), axis=1)

raw_data["polysemy"] = np.nan
raw_data["polysemy"] = raw_data.apply(lambda x: freq_of_words_great_sent_len(x['abstract']), axis=1)

# raw_data["flesch_reading_ease"] = np.nan
# raw_data["flesch_reading_ease"] = raw_data.apply(lambda x: flesch_reading_ease(x['abstract']), axis=1)

raw_data.head()

,abstract,review,semantic_similarity,abstract_clean,review_clean,semantic_similarity_aftercleaning,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs,polysemy,flesch_reading_ease
0,Distributional Semantics Models(DSM) derive wo...,I hate to say that the current version of this...,[[0.8928093]],distributional semantics modelsdsm derive wor...,hate current version paper ready poorly write...,[[0.83050203]],19.643612,6.035088,126.000000,121,121,NaN
1,Teaching plays a very important role in our so...,The authors define a deep learning model compo...,999999999,teaching play important role society spread h...,author define deep learn model compose compone...,[[0.87180996]],22.545755,5.706349,210.250000,112,112,NaN
2,Deep learning achieves remarkable generalizati...,The paper aims to provide a view of deep learn...,[[0.8619562]],deep learn achieve remarkable generalization c...,paper aim provide view deep learn perspective ...,[[0.8272942]],46.875000,6.619048,132.333333,57,57,NaN
3,Long Short-Term Memory (LSTM) is one of the mo...,"This paper propose a new ""gate"" function for L...",[[0.8917985]],long short term memory lstm widely use recu...,paper propose new gate function lstm enable ...,[[0.8526045]],21.028683,5.381579,180.875000,92,92,NaN
4,A core aspect of human intelligence is the ab...,The authors propose a kind of framework for le...,[[0.8901408]],core aspect human intelligence ability learn ...,author propose kind framework learn solve elem...,[[0.88526076]],41.541958,5.962733,159.428571,82,82,NaN


In [126]:
data = raw_data.copy()
data['similarity_score'] = pd.to_numeric(data['semantic_similarity_aftercleaning'].str.replace('[','').str.replace(']',''))
data = data[data['similarity_score'] != 999999999.0].copy()
data['similarity_score'] = data['similarity_score'] * 100

In [127]:
tf.word_count(data,"abstract","word_cnt")
tf.char_count(data,"abstract","char_len")
tf.avg_word_length(data,"abstract","avg_wrd_length")
tf.stopwords_count(data,"abstract","stopwords_cnt")
tf.numerics_count(data,"abstract","num_len")

,abstract,review,semantic_similarity,abstract_clean,review_clean,semantic_similarity_aftercleaning,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs,polysemy,flesch_reading_ease,similarity_score,word_cnt,char_len,avg_wrd_length,stopwords_cnt,num_len
0,Distributional Semantics Models(DSM) derive wo...,I hate to say that the current version of this...,[[0.8928093]],distributional semantics modelsdsm derive wor...,hate current version paper ready poorly write...,[[0.83050203]],19.643612,6.035088,126.000000,121,121,NaN,83.050203,260,2031,6.035088,88,0
1,Teaching plays a very important role in our so...,The authors define a deep learning model compo...,999999999,teaching play important role society spread h...,author define deep learn model compose compone...,[[0.87180996]],22.545755,5.706349,210.250000,112,112,NaN,87.180996,252,1689,5.706349,90,0
2,Deep learning achieves remarkable generalizati...,The paper aims to provide a view of deep learn...,[[0.8619562]],deep learn achieve remarkable generalization c...,paper aim provide view deep learn perspective ...,[[0.8272942]],46.875000,6.619048,132.333333,57,57,NaN,82.729420,105,799,6.619048,27,0
3,Long Short-Term Memory (LSTM) is one of the mo...,"This paper propose a new ""gate"" function for L...",[[0.8917985]],long short term memory lstm widely use recu...,paper propose new gate function lstm enable ...,[[0.8526045]],21.028683,5.381579,180.875000,92,92,NaN,85.260450,228,1454,5.381579,87,1
4,A core aspect of human intelligence is the ab...,The authors propose a kind of framework for le...,[[0.8901408]],core aspect human intelligence ability learn ...,author propose kind framework learn solve elem...,[[0.88526076]],41.541958,5.962733,159.428571,82,82,NaN,88.526076,163,1122,5.962733,49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,We examine how learning from unaligned data ca...,This work propose a generative model for unsup...,[[0.88786817]],examine learn unaligned datum improve datum ef...,work propose generative model unsupervised lea...,[[0.83547205]],30.311429,5.858824,165.571429,75,75,NaN,83.547205,170,1165,5.858824,57,0
2744,Deep Neural Networks (DNNs) have recently been...,This paper tried to analyze the subspaces of t...,[[0.82397217]],deep neural networks dnns recently vulnerabl...,paper try analyze subspace adversarial example...,[[0.7635975]],20.751118,6.188406,211.571429,87,87,NaN,76.359750,207,1487,6.188406,75,0
2745,The success of Deep Learning and its potential...,The paper studies methods for verifying neural...,[[0.90048885]],success deep learning potential use important ...,paper study method verify neural net piecewise...,[[0.8841833]],25.371742,5.493506,188.750000,100,100,NaN,88.418330,213,1517,5.493506,94,0
2746,We introduce a new dataset of logical entailme...,This is a wonderful and a self-contained paper...,[[0.83777016]],introduce new dataset logical entailments purp...,wonderful self contain paper fact introduce...,[[0.83051616]],28.195122,6.058824,239.000000,44,44,NaN,83.051616,102,719,6.058824,38,0


In [128]:
columns = ['yules_i_measure_abs', 'avg_word_len_abs', 'avg_sen_len_abs', 'freq_words_gt_sen_len_abs', 'similarity_score',
          'word_cnt', 'char_len', 'avg_wrd_length', 'stopwords_cnt', 'num_len', 'polysemy']
df = data[columns].copy()
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
#df = pd.DataFrame(x_scaled)
df=pd.DataFrame(x_scaled, columns=df.columns)

In [129]:
training_set = data.sample(frac=0.8, random_state = 1)
test_set = data.loc[~data.index.isin(training_set.index)]
test_set.head()

training_set = df.sample(frac=0.8, random_state = 1)
test_set = df.loc[~df.index.isin(training_set.index)]
test_set.head()

,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs,similarity_score,word_cnt,char_len,avg_wrd_length,stopwords_cnt,num_len,polysemy
14,0.238270,0.378058,0.434597,0.449664,0.901918,0.415430,0.439834,0.378058,0.358209,0.00,0.449664
15,0.181193,0.529975,0.571472,0.483221,0.798186,0.412463,0.536192,0.529975,0.350746,0.25,0.483221
18,0.421369,0.530478,0.228301,0.342282,0.647917,0.359050,0.408944,0.530478,0.313433,0.00,0.342282
20,0.234547,0.423638,0.681579,0.731544,0.809670,0.670623,0.716459,0.423638,0.529851,0.00,0.731544
25,0.115274,0.809900,0.608802,0.449664,0.650885,0.367953,0.467036,0.809900,0.298507,0.00,0.449664


In [130]:
data_columns = ['yules_i_measure_abs', 'avg_word_len_abs',
       'avg_sen_len_abs', 'freq_words_gt_sen_len_abs', 'polysemy']

columns = ['yules_i_measure_abs', 'avg_word_len_abs', 'avg_sen_len_abs', 'freq_words_gt_sen_len_abs',
          'word_cnt', 'char_len', 'avg_wrd_length', 'stopwords_cnt', 'num_len', 'polysemy']


# training_data = training_set.as_matrix(columns = data_columns)
training_data = training_set[data_columns].to_numpy()# .as_matrix(columns = data_columns)
nan_locs = np.isnan(training_data)
training_data[nan_locs] = 0

training_target = training_set['similarity_score'].values
nan_locs = np.isnan(training_target)
training_target[nan_locs] = 0

test_data = test_set[data_columns].to_numpy()# .as_matrix(columns = data_columns)
nan_locs = np.isnan(test_data)
test_data[nan_locs] = 0

test_target = test_set['similarity_score'].values
nan_locs = np.isnan(test_target)
test_target[nan_locs] = 0

In [131]:
train_data_array = training_data.copy()
train_class_array = training_target.copy()
test_data_array = test_data.copy()
test_class_array = test_target.copy()

In [132]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor()
reg.fit(training_data, training_target)

print('R squared: ' + str(reg.score(test_data, test_target)))
pred = reg.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.08545539471761376
MAE: 0.0902223422989583
MSE: 0.013266340136723107


In [133]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(training_data, training_target)

print('R squared: ' + str(svr.score(test_data, test_target)))
pred = svr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.06118514264576225
MAE: 0.09118600136808633
MSE: 0.013618403248056831


In [134]:
from sklearn.kernel_ridge import KernelRidge
kr = KernelRidge()
kr.fit(training_data, training_target)

print('R squared: ' + str(kr.score(test_data, test_target)))
pred = kr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -1.070930251245128
MAE: 0.13903085048855154
MSE: 0.03004081479870978


In [135]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(training_data, training_target)

print('R squared: ' + str(dt.score(test_data, test_target)))
pred = dt.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.28144660267259747
MAE: 0.10402708935663606
MSE: 0.018588602895813683


In [136]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 2000)
rf.fit(training_data, training_target)

print('R squared: ' + str(rf.score(test_data, test_target)))
pred = rf.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.05002391424552077
MAE: 0.09518650660501492
MSE: 0.015231596488148614


In [137]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
gb.fit(training_data, training_target)

print('R squared: ' + str(gb.score(test_data, test_target)))
pred = gb.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.10565330162846309
MAE: 0.08921293779777568
MSE: 0.012973350268780623


In [138]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pr = Pipeline([('poly', PolynomialFeatures(degree = 3)),
              ('linear', LinearRegression(fit_intercept = False))])

pr.fit(training_data, training_target)

print('R squared: ' + str(pr.score(test_data, test_target)))
pred = pr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.0834575754813025
MAE: 0.09000351718415334
MSE: 0.013295320406649262


In [139]:
from sklearn.linear_model import Lasso
lm = Lasso()
lm.fit(training_data, training_target)

print('R squared: ' + str(lm.score(test_data, test_target)))
pred = lm.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.0006330899803945744
MAE: 0.09644034345586966
MSE: 0.014515135562623868


In [140]:
from sklearn.linear_model import ElasticNet
en = ElasticNet()
en.fit(training_data, training_target)

print('R squared: ' + str(en.score(test_data, test_target)))
pred = en.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.0006330899803945744
MAE: 0.09644034345586966
MSE: 0.014515135562623868


In [141]:
from sklearn.linear_model import LassoLars
ll = LassoLars()
ll.fit(training_data, training_target)

print('R squared: ' + str(ll.score(test_data, test_target)))
pred = ll.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.0006330899803945744
MAE: 0.09644034345586966
MSE: 0.014515135562623868


In [142]:
from sklearn.linear_model import BayesianRidge
br = BayesianRidge()
br.fit(training_data, training_target)

print('R squared: ' + str(br.score(test_data, test_target)))
pred = br.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.056345437170144175
MAE: 0.09239987453181636
MSE: 0.01368860778333074


In [143]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor()
sgd.fit(training_data, training_target)

print('R squared: ' + str(sgd.score(test_data, test_target)))
pred = sgd.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.04704854412818005
MAE: 0.09822602704467151
MSE: 0.015188435912074692


In [ ]:
\begin{table}[ht]
\caption{Model Results} % title of Table
\centering % used for centering table
\begin{tabular}{c c c c} % centered columns (4 columns)
\hline\hline %inserts double horizontal lines
Case & Method\#1 & Method\#2 & Method\#3 \\ [0.5ex] % inserts table
%heading
\hline % inserts single horizontal line
1 & 50 & 837 & 970 \\ % inserting body of the table
2 & 47 & 877 & 230 \\
3 & 31 & 25 & 415 \\
4 & 35 & 144 & 2356 \\
5 & 45 & 300 & 556 \\ [1ex] % [1ex] adds vertical space
\hline %inserts single line
\end{tabular}
\label{table:nonlin} % is used to refer this table in the text
\end{table}
--------------------- e

In [144]:
from sklearn.model_selection import RandomizedSearchCV
# Setup the parameters and distributions to sample from: param_dist
param_dist_dt = {"max_depth": [1, 32],
              "max_features": list(range(1, 21)),
              "random_state": list(range(1, 5000)),
              "min_samples_split": np.linspace(0.1, 1, 10, endpoint=True),
              "min_samples_leaf": np.linspace(0.1, 0.5, 5, endpoint=True),
              "criterion": ["friedman_mse", "mse"]}
## Instantiate the RandomizedSearchCV object: tree_cv
dt_cv = RandomizedSearchCV(DecisionTreeRegressor(), \
                           param_dist_dt, cv=10, n_jobs=-1, verbose=1)
# Fit it to the data
dt_cv.fit(train_data_array, train_class_array)
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(dt_cv.best_params_))
print("Best score is {}".format(dt_cv.best_score_))
# train mse
pred = dt_cv.predict(train_data_array)
score = metrics.mean_squared_error(train_class_array, pred)
print("Train MSE:   %0.3f" % score)
# test mse
pred = dt_cv.predict(test_data_array)
score = metrics.mean_squared_error(test_class_array, pred)
print("Test MSE:   %0.3f" % score)
# test mae
pred = dt_cv.predict(test_data_array)
score = metrics.mean_absolute_error(test_class_array, pred)
print("Test MAE:   %0.3f" % score)
# r-squared
score = metrics.r2_score(test_class_array, pred)
print("R-squared  %0.3f" % score)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Tuned Decision Tree Parameters: {'random_state': 1124, 'min_samples_split': 0.1, 'min_samples_leaf': 0.2, 'max_features': 4, 'max_depth': 32, 'criterion': 'friedman_mse'}
Best score is 0.08582546032863245
Train MSE:   0.013
Test MSE:   0.013
Test MAE:   0.091
R-squared  0.095


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.7s finished


In [145]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"learning_rate": [1, 0.5, 0.25, 0.1, 0.05, 0.01],
              "n_estimators": [1, 2, 4, 8, 16, 32, 64, 100, 200],
              "random_state": list(range(1, 5000)),
              "max_features": list(range(1, 9)),
              "max_depth": np.linspace(1, 32, 32, endpoint=True),
              "min_samples_split": np.linspace(0.1, 1.0, 10, endpoint=True),
              "min_samples_leaf": np.linspace(0.1, 0.5, 5, endpoint=True)}
# Instantiate a Decision Tree classifier: tree
gdb = GradientBoostingRegressor()
# Instantiate the RandomizedSearchCV object: tree_cv
gdb_cv = RandomizedSearchCV(gdb, param_dist, cv=10, n_jobs=-1, verbose=1)
# Fit it to the data
gdb_cv.fit(train_data_array, train_class_array)
# Print the tuned parameters and score
print("Tuned Gradient Boosting Parameters: {}".format(gdb_cv.best_params_))
print("Best score is {}".format(rf_cv.best_score_))
# train mse
pred = gdb_cv.predict(train_data_array)
score = metrics.mean_squared_error(train_class_array, pred)
print("Train MSE:   %0.3f" % score)
# test mse
pred = gdb_cv.predict(test_data_array)
score = metrics.mean_squared_error(test_class_array, pred)
print("Test MSE:   %0.3f" % score)
# test mae
pred = gdb_cv.predict(test_data_array)
score = metrics.mean_absolute_error(test_class_array, pred)
print("Test MAE:   %0.3f" % score)
# r-squared
score = metrics.r2_score(test_class_array, pred)
print("R-squared  %0.3f" % score)

OverflowError: Python int too large to convert to C long